# Simulação da Carga para Camadas Gold e Platinnum
## Configuração do ambiente

In [1]:
import time
from datetime import date, timedelta
from delta import *
from os import path
from pyspark.sql.functions import translate, to_timestamp, date_format,concat_ws, col
from pyspark.sql.types import StringType, DateType, StructType, IntegerType, LongType

In [2]:

spark.conf.set(f'fs.azure.account.auth.type.{STORAGE_ACCOUNT}.dfs.core.windows.net', 'OAuth')
spark.conf.set(f'fs.azure.account.oauth.provider.type.{STORAGE_ACCOUNT}.dfs.core.windows.net',  'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider')
spark.conf.set(f'fs.azure.account.oauth2.client.id.{STORAGE_ACCOUNT}.dfs.core.windows.net', APPLICATION_ID)
spark.conf.set(f'fs.azure.account.oauth2.client.secret.{STORAGE_ACCOUNT}.dfs.core.windows.net', PASSWORD)
spark.conf.set(f'fs.azure.account.oauth2.client.endpoint.{STORAGE_ACCOUNT}.dfs.core.windows.net', f'https://login.microsoftonline.com/{DIRECTORY_ID}/oauth2/token')

Define os caminhos para acessar as camadas bronze, silver e gold além de definir o schema para a camada gold. Caso esta ainda não exista, criar a Delta Table para a camada

In [3]:
bronze_table = f'abfss://{LAKEHOUSE}@{STORAGE_ACCOUNT}.dfs.core.windows.net/bronze/{TOPIC}'
silver_table = f'abfss://{LAKEHOUSE}@{STORAGE_ACCOUNT}.dfs.core.windows.net/silver/{TOPIC}'
gold_table = f'abfss://{LAKEHOUSE}@{STORAGE_ACCOUNT}.dfs.core.windows.net/gold/{TOPIC}'
checkpoint_path = f'abfss://{LAKEHOUSE}@{STORAGE_ACCOUNT}.dfs.core.windows.net/checkpoint/silver/delta/{TOPIC}'

if not DeltaTable.isDeltaTable(spark, silver_table):
    print(f'Criar tabela {TOPIC}')
    if TOPIC == 'audit':
        schema = (StructType()
            .add('_id', StringType())
            .add('ano', IntegerType())
            .add('mes', IntegerType())
            .add('dia', IntegerType())
            .add('hora', IntegerType())
            .add('minuto', IntegerType())                  
            .add('idacao', StringType())
            .add('idproduto', StringType())
            .add('idservico', StringType())
            .add('idatividade', StringType())
            .add('payload', StringType())
            .add('porta', StringType())
            .add('idusuario', StringType())
            .add('nomeusuario', StringType())
            .add('localizador_tipo', StringType())
            .add('localizador', StringType())
            .add('ip', StringType())
            .add('data_fim', StringType())
            .add('data_inicio', StringType())
            .add('idcliente', StringType())
            .add('cliente', StringType())
            .add('idplataforma', StringType())
            .add('plataforma', StringType())
            .add('data', StringType())
            .add('horario', StringType())
            .add('data_criacao', StringType())
            .add('duracao', StringType())
            .add('produto', StringType())
            .add('atividade', StringType())
            .add('acao', StringType())
            .add('servico', StringType())
        )
        emptyDF = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
        emptyDF.write.format('delta').mode('overwrite').partitionBy('ano', 'mes', 'dia').save(silver_table)        
    else:
        schema = (StructType()
            .add('_id', StringType())
            .add('nome', StringType())
        )    
        emptyDF = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
        emptyDF.write.format('delta').mode('overwrite').save(silver_table)
    

deltaTable = DeltaTable.forPath(spark, silver_table)

In [4]:
def carregar_tabela(tabela, tier='bronze'):
#     df = spark.read.format('delta').load(path.join(LAKEHOUSE, 'bronze', tabela))
    df = spark.read.format('delta').load(f'abfss://{LAKEHOUSE}@{STORAGE_ACCOUNT}.dfs.core.windows.net/{tier}/{tabela}') \

    df.createOrReplaceTempView(tabela)
    return df

def config_upsert(delta):
    def upsertToDelta(microbatchdf, batchId):
        delta.alias("t").merge(
          microbatchdf.alias("s"),
          "s._id = t._id") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
        print(f'Exportadas {microbatchdf.count()} linhas')
        
    return upsertToDelta

In [5]:
%%time
# df = (
#     spark
#     .readStream
#     .format('delta')
#     .option('startingOffsets', 'latest')
#     .load(bronze_table)
#     .withColumn('data_criacao', to_timestamp('data_criacao'))
#     .withColumn('ano', date_format('data_criacao', 'yyyy').cast(IntegerType()))
#     .withColumn('mes', date_format('data_criacao', 'MM').cast(IntegerType()))
#     .withColumn('dia', date_format('data_criacao', 'dd').cast(IntegerType()))
#     .withColumn('hora', date_format('data_criacao', 'HH').cast(IntegerType()))
#     .withColumn('minuto', date_format('data_criacao', 'mm').cast(IntegerType()))    
#     .writeStream
#     .format('delta')
#     .foreachBatch(config_upsert(deltaTable))
#     .outputMode('update')
#     .option('checkpointLocation', checkpoint_path)
#     .trigger(once=True)
#     .start()
#     .awaitTermination()
# )

# 29140896

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.15 µs


In [6]:
%%time
df_bronze = carregar_tabela(TOPIC, 'bronze')
# df_bronze.toPandas()

CPU times: user 1.7 ms, sys: 1.61 ms, total: 3.31 ms
Wall time: 2.35 s


In [7]:
%%time
df_silver = carregar_tabela(TOPIC, 'silver')
# df_silver.toPandas()

CPU times: user 1.38 ms, sys: 1.49 ms, total: 2.87 ms
Wall time: 192 ms


In [8]:
%%time
print(f'Bronze: {df_bronze.count()}')
print(f'Silver: {df_silver.count()}')

# 07:07
# Bronze: 28909691
# Silver: 28909691

# 11:00
# Bronze: 28909801
# Silver: 28909801

# Bronze: 29140896
# Silver: 29140896

# Bronze: 29189458
# Silver: 29189289

# 29.730.484 9:27
# 29.741.472
# 29.919.115
# 29.946.055

Bronze: 29946878
Silver: 29946878
CPU times: user 7.43 ms, sys: 4.54 ms, total: 12 ms
Wall time: 1min 9s


### Processar GOLD

In [9]:
df_silver.printSchema()
df_silver.createOrReplaceTempView("evento")

root
 |-- _id: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- minuto: integer (nullable = true)
 |-- idacao: string (nullable = true)
 |-- idproduto: string (nullable = true)
 |-- idservico: string (nullable = true)
 |-- idatividade: string (nullable = true)
 |-- payload: string (nullable = true)
 |-- porta: string (nullable = true)
 |-- idusuario: string (nullable = true)
 |-- nomeusuario: string (nullable = true)
 |-- localizador_tipo: string (nullable = true)
 |-- localizador: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- data_fim: string (nullable = true)
 |-- data_inicio: string (nullable = true)
 |-- idcliente: string (nullable = true)
 |-- cliente: string (nullable = true)
 |-- idplataforma: string (nullable = true)
 |-- plataforma: string (nullable = true)
 |-- data: string (nullable = true)
 |-- horario: string (nullable = true)
 |--

In [10]:
hoje = date.today()
ontem = hoje - timedelta(1)
print(f'Hoje..: {hoje}')
print(f'Ontem.: {ontem}')

Hoje..: 2022-03-21
Ontem.: 2022-03-20


In [11]:
if not DeltaTable.isDeltaTable(spark, gold_table):
    print(f'Criar tabela {TOPIC}')
    schema = (StructType()
        .add('ano', IntegerType())
        .add('mes', IntegerType())
        .add('dia', IntegerType())
        .add('hora', IntegerType())
        .add('minuto', IntegerType())                  
        .add('idplataforma', StringType())
        .add('plataforma', StringType())
        .add('idcliente', StringType())
        .add('cliente', StringType())
        .add('idservico', StringType())
        .add('servico', StringType())
        .add('idproduto', StringType())
        .add('produto', StringType())              
        .add('idatividade', StringType())
        .add('atividade', StringType())
        .add('idacao', StringType())
        .add('acao', StringType())                      
        .add('idusuario', StringType())
        .add('usuario', StringType())
        .add('quantidade', LongType())              
    )
    emptyDF = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)
    emptyDF.write.format('delta').mode('overwrite').partitionBy('ano', 'mes', 'dia').save(gold_table)        


delta_gold = DeltaTable.forPath(spark, gold_table)

Criar tabela audit


In [12]:
df_gold = spark.read.format('delta').load(gold_table)
df_gold.printSchema()

root
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- minuto: integer (nullable = true)
 |-- idplataforma: string (nullable = true)
 |-- plataforma: string (nullable = true)
 |-- idcliente: string (nullable = true)
 |-- cliente: string (nullable = true)
 |-- idservico: string (nullable = true)
 |-- servico: string (nullable = true)
 |-- idproduto: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- idatividade: string (nullable = true)
 |-- atividade: string (nullable = true)
 |-- idacao: string (nullable = true)
 |-- acao: string (nullable = true)
 |-- idusuario: string (nullable = true)
 |-- usuario: string (nullable = true)
 |-- quantidade: long (nullable = true)



In [13]:
%%time
df = spark.sql(f'''
select
    ano,
    mes,
    dia,
    hora,
    minuto,
    idplataforma,
    idcliente,
    idservico,
    idproduto,    
    idatividade,
    idacao,
    idusuario,
    count(*) as quantidade
from evento
-- where ano in ({hoje.year}, {ontem.year})
--  and mes in ({hoje.month}, {ontem.month})
--  and dia in ({hoje.day}, {ontem.day})
group by
    ano,
    mes,
    dia,
    hora,
    minuto,
    idplataforma,
    idcliente,
    idservico,
    idproduto,  
    idatividade,
    idacao,
    idusuario
order by
    ano,
    mes,
    dia,
    hora,
    minuto,
    idplataforma,
    idcliente,
    idservico,
    idproduto,  
    idatividade,
    idacao,
    idusuario
''')

df.createOrReplaceTempView('estatistica')

df.toPandas()

CPU times: user 7.79 s, sys: 599 ms, total: 8.39 s
Wall time: 1min 33s


,ano,mes,dia,hora,minuto,idplataforma,idcliente,idservico,idproduto,idatividade,idacao,idusuario,quantidade
0,2020,2,27,21,8,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f83ce7d620f184d444b,5e4c555ad52d1c182dc1cb8f,5b563a26a72acc7d1e169176,1
1,2020,2,27,21,8,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f83ce7d620f184d444b,5e4c555ad52d1c182dc1cb8f,5e3af6682ec70521bd72882b,31
2,2020,2,27,21,8,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f83ce7d620f184d444b,5e4c555ad52d1c182dc1cb8f,5e42c695982952ebca48ae10,8
3,2020,2,27,21,8,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f83ce7d620f184d444b,5e4c555ad52d1c182dc1cb8f,5e5017126fa8736e01b191a3,2
4,2020,2,27,21,9,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f83ce7d620f184d444b,5e4c555ad52d1c182dc1cb8f,5b563a26a72acc7d1e169176,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131653,2022,3,20,20,8,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f66ce7d620f184d4449,5e4c555ad52d1c182dc1cb90,6101c33703b5809d7d5ef127,1
1131654,2022,3,20,20,10,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f66ce7d620f184d4449,5e4c555ad52d1c182dc1cb90,6101c33703b5809d7d5ef127,2
1131655,2022,3,20,20,11,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f66ce7d620f184d4449,5e4c555ad52d1c182dc1cb90,6101c33703b5809d7d5ef127,1
1131656,2022,3,20,20,13,5e455d76ce7d620f184d443d,5e455cb8ce7d620f184d443c,5e455ed9ce7d620f184d4446,5e455e09ce7d620f184d4442,5e455f66ce7d620f184d4449,5e4c555ad52d1c182dc1cb90,6101c33703b5809d7d5ef127,1


#### Gerar tabelas de apoio

In [14]:
%%time
df_plataforma = spark.sql('select distinct idplataforma, plataforma from evento')
df_plataforma.createOrReplaceTempView('plataforma')
df_cliente = spark.sql('select distinct idcliente, cliente from evento')
df_cliente.createOrReplaceTempView('cliente')
df_servico = spark.sql('select distinct idservico, servico from evento')
df_servico.createOrReplaceTempView('servico')
df_produto = spark.sql('select distinct idproduto, produto from evento')
df_produto.createOrReplaceTempView('produto')
df_atividade = spark.sql('select distinct idatividade, atividade from evento')
df_atividade.createOrReplaceTempView('atividade')
df_acao = spark.sql('select distinct idacao, acao from evento')
df_acao.createOrReplaceTempView('acao')

CPU times: user 2.45 ms, sys: 1.31 ms, total: 3.76 ms
Wall time: 55.1 ms


In [15]:
df_usuario = spark.sql('''
select distinct 
    idusuario, 
    nomeusuario as usuario 
from evento 
order by 1
''')
df_usuario = df_usuario.dropDuplicates(['idusuario'])
df_usuario.createOrReplaceTempView('usuario')

df_usuario.toPandas()

,idusuario,usuario
0,5b5639a3a72acc7d1e169175,Bruno Aurelio
1,5b563a26a72acc7d1e169176,admin
2,5b9ab33a00d165923ca5beee,5b9ab33a00d165923ca5beee
3,5de944fda1e187078a73875f,Bianca A de Oliveira
4,5e260884ec3807463e32afa3,Administrador BF
...,...,...
1403,622f419653ba83a25cb1ffcc,Comercial
1404,6233338ad8b02d963be4208a,BARBARA LETICIA PASSOS DOS SANTOS
1405,6233474190f10ddf4ccdccf0,fabioc
1406,6234a2f59cf744683094de60,JESSICA ARIANE OLIVEIRA MARCIANO


In [16]:
spark.sql('''
select *
from usuario
where idusuario = "5b563a26a72acc7d1e169176"
''').toPandas()

,idusuario,usuario
0,5b563a26a72acc7d1e169176,admin


In [17]:
%%time
df = spark.sql('''
select
    e.ano,
    e.mes,
    e.dia,
    e.hora,
    e.minuto,
    e.idplataforma,
    p.plataforma,
    e.idcliente,
    c.cliente,
    e.idservico,
    s.servico,
    e.idproduto,
    pd.produto,
    e.idatividade,
    a.atividade,
    e.idacao,
    ac.acao,
    e.idusuario,
    u.usuario,
    e.quantidade
from estatistica e
    inner join plataforma p on p.idplataforma = e.idplataforma
    inner join cliente c on c.idcliente = e.idcliente
    inner join servico s on s.idservico = e.idservico
    inner join produto pd on pd.idproduto = e.idproduto
    inner join atividade a on a.idatividade = e.idatividade
    inner join acao ac on ac.idacao = e.idacao
    inner join usuario u on u.idusuario = e.idusuario
order by
    e.ano,
    e.mes,
    e.dia,
    e.hora,
    e.minuto,
    e.idplataforma,
    e.idcliente,
    e.idservico,
    e.idproduto,    
    e.idatividade,
    e.idacao,
    e.idusuario,
    e.quantidade    
''')
df.printSchema()

root
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- minuto: integer (nullable = true)
 |-- idplataforma: string (nullable = true)
 |-- plataforma: string (nullable = true)
 |-- idcliente: string (nullable = true)
 |-- cliente: string (nullable = true)
 |-- idservico: string (nullable = true)
 |-- servico: string (nullable = true)
 |-- idproduto: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- idatividade: string (nullable = true)
 |-- atividade: string (nullable = true)
 |-- idacao: string (nullable = true)
 |-- acao: string (nullable = true)
 |-- idusuario: string (nullable = true)
 |-- usuario: string (nullable = true)
 |-- quantidade: long (nullable = false)

CPU times: user 1.53 ms, sys: 1.9 ms, total: 3.42 ms
Wall time: 65.2 ms


In [18]:
%%time
df.write.format('delta').mode('append').save(gold_table)

CPU times: user 35.7 ms, sys: 23.4 ms, total: 59.1 ms
Wall time: 5min 55s


### Ler e gravar o DF no Yugabyte

In [ ]:
# df_gold = (spark.read
#     .format('delta')
#     .load(gold_table)
#     .where('ano = 2020')
#     .withColumn('data_base', concat_ws('-', col('ano'), col('mes'), col('dia')))
#     .withColumn('hora_base', concat_ws(':', col('hora'), col('minuto')))
#     .withColumn('horario', concat_ws(' ', col('data_base'), col('hora_base')))
#     .withColumn('horario', to_timestamp('horario'))
#     .drop('data_base')
#     .drop('hora_base')
# )
# df_gold.count()
# df_gold.printSchema()
# df_gold.select('data_base', 'hora_base', 'horario').limit(10).toPandas()

In [ ]:
# (df_gold.write
#  .mode('append')
#  .format('jdbc')
#  .option("url", "jdbc:postgresql://20.201.79.89:5433/amnis")
#  .option("driver", "org.postgresql.Driver")
#  .option("dbtable", "eventos")
#  .option("user", "admin")
#  .option("password", "Zaq12wsX")
#  .save()
# )

In [19]:
for ano in [2020, 2021, 2022]:
    for mes in range(1,13):
        print(f'Processando {mes}/{ano}')
        gravou = False
        for dia in range(1,32):
            df_gold = (spark.read
                .format('delta')
                .load(gold_table)
                .where(f'ano = {ano} and mes = {mes} and dia = {dia}')
                .withColumn('data_base', concat_ws('-', col('ano'), col('mes'), col('dia')))
                .withColumn('hora_base', concat_ws(':', col('hora'), col('minuto')))
                .withColumn('horario', concat_ws(' ', col('data_base'), col('hora_base')))
                .withColumn('horario', to_timestamp('horario'))
                .drop('data_base')
                .drop('hora_base')
            )

            registros = df_gold.count()

            if registros > 0:
                gravou = True
                print(f'  {dia} gravar {registros} registros')
                (df_gold.write
                 .mode('append')
                 .format('jdbc')
                 .option("url", "jdbc:postgresql://20.201.79.89:5433/amnis")
                 .option("driver", "org.postgresql.Driver")
                 .option("dbtable", "eventos")
                 .option("user", "admin")
                 .option("password", "Zaq12wsX")
                 .save()
                ) 
        if gravou:
            time.sleep(60)
        

Processando 1/2020
Processando 2/2020
  27 gravar 15 registros
  28 gravar 21 registros
  29 gravar 15 registros
Processando 3/2020
  2 gravar 32 registros
  3 gravar 24 registros
  4 gravar 31 registros
  5 gravar 29 registros
  6 gravar 6 registros
  7 gravar 16 registros
  9 gravar 36 registros
  10 gravar 41 registros
  11 gravar 23 registros
  12 gravar 82 registros
  13 gravar 37 registros
  15 gravar 1 registros
  16 gravar 49 registros
  17 gravar 24 registros
  18 gravar 27 registros
  19 gravar 11 registros
  20 gravar 41 registros
  21 gravar 21 registros
  23 gravar 50 registros
  24 gravar 28 registros
  25 gravar 11 registros
  26 gravar 21 registros
  27 gravar 17 registros
  30 gravar 153 registros
  31 gravar 90 registros
Processando 4/2020
  1 gravar 49 registros
  2 gravar 41 registros
  3 gravar 6 registros
  4 gravar 3 registros
  6 gravar 24 registros
  7 gravar 10 registros
  8 gravar 77 registros
  9 gravar 68 registros
  10 gravar 27 registros
  12 gravar 9 reg

  25 gravar 1090 registros
  26 gravar 1215 registros
  27 gravar 1261 registros
  28 gravar 1624 registros
  29 gravar 1526 registros
  30 gravar 537 registros
  31 gravar 43 registros
Processando 2/2021
  1 gravar 1213 registros
  2 gravar 1303 registros
  3 gravar 1866 registros
  4 gravar 1559 registros
  5 gravar 1686 registros
  6 gravar 112 registros
  7 gravar 31 registros
  8 gravar 1574 registros
  9 gravar 1179 registros
  10 gravar 1982 registros
  11 gravar 1657 registros
  12 gravar 2431 registros
  13 gravar 674 registros
  14 gravar 16 registros
  15 gravar 1180 registros
  16 gravar 1149 registros
  17 gravar 2031 registros
  18 gravar 1907 registros
  19 gravar 2249 registros
  20 gravar 145 registros
  22 gravar 981 registros
  23 gravar 1569 registros
  24 gravar 1781 registros
  25 gravar 1367 registros
  26 gravar 1849 registros
  27 gravar 357 registros
  28 gravar 244 registros
Processando 3/2021
  1 gravar 1497 registros
  2 gravar 1904 registros
  3 gravar 170

  1 gravar 1905 registros
  2 gravar 2014 registros
  3 gravar 2104 registros
  4 gravar 1322 registros
  5 gravar 230 registros
  6 gravar 1396 registros
  7 gravar 1853 registros
  8 gravar 2476 registros
  9 gravar 2735 registros
  10 gravar 1338 registros
  11 gravar 1081 registros
  12 gravar 397 registros
  13 gravar 1653 registros
  14 gravar 1723 registros
  15 gravar 1685 registros
  16 gravar 1998 registros
  17 gravar 1606 registros
  18 gravar 336 registros
  19 gravar 33 registros
  20 gravar 1355 registros
  21 gravar 1354 registros
  22 gravar 2385 registros
  23 gravar 2252 registros
  24 gravar 518 registros
  25 gravar 3 registros
  26 gravar 47 registros
  27 gravar 1513 registros
  28 gravar 2085 registros
  29 gravar 2759 registros
  30 gravar 1820 registros
  31 gravar 248 registros
Processando 1/2022
  1 gravar 11 registros
  2 gravar 40 registros
  3 gravar 1485 registros
  4 gravar 1445 registros
  5 gravar 2499 registros
  6 gravar 461 registros
  7 gravar 754